<a href="https://colab.research.google.com/github/NataliaRusinchuk/DigiLabStar/blob/main/Chua_circuit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ctypes import c_int32
import math
from typing import Callable, List, Tuple, Union
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

Value = Union[float, int]
Vector = Union[List[Value], np.ndarray, Tuple[Value, ...]]


class ODEsystem:
    """Class for first order differential equation"""

    def __init__(
        self,
        t0: Value,
        x0: Vector,
        f_tuple: Tuple[Callable[[Value, Vector], float], ...],
    ):
        """
        Initialization of the class instance
        ---
        Parameters:
        --
        t0 :  float | int
                initial time
        x0 :  list[float | int]
                initial values of the functions in the system
        f :   tuple[Callable[[list[Value]], float], ...]
                tuple of functions f(t,X) from ODE system f'(X)=f(t,X)
                X is a vector of unknown functions
        """
        self.t0 = t0
        self.x0 = x0
        self.equations = f_tuple
        self.size = len(x0)

    def __calc_funcs(self, t: Value, X: Vector) -> np.ndarray:
        """
        Returns np.ndarray with value
        of all the functions of differential equations
        """
        funcs = []
        for i in range(self.size):
            funcs += [self.equations[i](t, X)]
        return np.array(funcs)

    def runge_4(self, tn: Value, h: Value = 0.001) -> Tuple[np.ndarray, np.ndarray]:
        """
        4th order Runge-Kutta method
        for numerical solution of the system of equations

        Returns tuple of np.adarrays for time values and function values
        ---
        Parameters
        --
        tn :    float | int
                end time point
        h :     float
                step for numerical solution
        --
        Returns
        --
        (t, x) :  tuple[np.ndarray, np.ndarray]
                t - array of time points of numerical solution
                x - array of numerical solution
                dimension of the array is (m, n)
                m is number of equations in the system
                n is number of time point of the solution
                x[i] is an array of values of i-th function for all time points
        """
        n = math.ceil((tn - self.t0) / h)
        t = np.arange(self.t0, tn + h, h)
        x = np.zeros((self.size, n + 1))
        x[:, 0] = self.x0
        for i in range(n):
            k1 = self.__calc_funcs(t[i], x[:, i])
            k2 = self.__calc_funcs(t[i] + h / 2, x[:, i] + h * k1 / 2)
            k3 = self.__calc_funcs(t[i] + h / 2, x[:, i] + h * k2 / 2)
            k4 = self.__calc_funcs(t[i] + h, x[:, i] + h * k3)
            x[:, i + 1] = x[:, i] + (k1 + 2 * k2 + 2 * k3 + k4) * h / 6
        return (t, x)

    def plot_solution(self, tn: Value, h: Value = 0.001):
        """
        Method for plotting the numerical solution of the equation
        ---
        Parameters
        tn :    float | int
                end time point
        h :     float
                step for numerical solution
        """
        t, x = self.runge_4(tn, h)
        for i in range(self.size):
            plt.plot(t, x[i], label=f"x[{i+1}]")
        plt.grid()
        plt.title("Time evolution of the system")
        plt.xlabel("Time")
        plt.ylabel("Functions")
        plt.legend()

    def phase_plane(self, tn: Value, i: int, j: int, h: Value = 0.001):
        """
        Method for plotting the set of numerical solutions of the equation in phase plane
        ---
        Parameters
        tn :    float | int
                end time point
        i :     int
                number of function to plot at x axis
        j :     int
                number of function to plot at y axis
        h :     float
                step for numerical solution
        """
        t, x = self.runge_4(tn, h)
        plt.plot(x[i], x[j])
        plt.grid()
        plt.title("Phase plane")
        plt.xlabel(f"x[{i}]")
        plt.ylabel(f"x[{j}]")

    def phase_portret(
        self,
        tn: Value,
        i: int,
        j: int,
        x1_set: Vector,
        x2_set: Vector,
        h: Value = 0.001,
    ):
        """
        Method for plotting the numerical solution of the equation in phase plane
        ---
        Parameters
        tn :    float | int
                end time point
        i :     int
                number of function to plot at x axis
        j :     int
                number of function to plot at y axis
        h :     float
                step for numerical solution
        """
        for x1_0 in x1_set:
            self.x0[i] = x1_0
            for x2_0 in x2_set:
                self.x0[j] = x2_0
                t, x = self.runge_4(tn, h)
                plt.plot(x[i], x[j])
                plt.grid()
                plt.title("Phase plane")
                plt.xlabel(f"x[{i}]")
                plt.ylabel(f"x[{j}]")
                n = int(len(x[i]) * 0.9)
                #'''
                plt.arrow(
                    x[i, n],
                    x[j, n],
                    x[i, n + 2] - x[i, n],
                    x[j, n + 2] - x[j, n],
                    head_width=0.01 * max(max(x[i]), max(x[j])),
                )
                #'''

    def phase_3d(self, tn: Value, i: int = 0, j: int = 1, k: int = 2, h: Value = 0.001):
        """
        Method for plotting the numerical solution of the equation in phase space
        ---
        Parameters
        tn :    float | int
                end time point
        i :     int
                number of function to plot at x axis
        j :     int
                number of function to plot at y axis
        k :     int
                number of function to plot at z axis
        h :     float
                step for numerical solution
        """
        if self.size < 3:
            print("Not acceptable, system has less than 3 equations")
        else:
            t, x = self.runge_4(tn, h)
            fig = plt.figure()
            ax = fig.add_subplot(projection="3d")
            ax.plot(x[i], x[j], x[k])
            ax.set_xlabel(f"x[{i}]")
            ax.set_ylabel(f"x[{j}]")
            ax.set_zlabel(f"x[{k}]")
            ax.set_title("Phase space")

    def phase_anim(
        self,
        tn: Value,
        filename: str,
        n: int = 0,
        j: int = 1,
        h: Value = 0.001,
        step: int = 500,
    ):
        """
        Method for creation of the animation of the numerical solution of the equation in phase plane
        ---
        Parameters
        tn :    float | int
                end time point
        filename: str
                name of the file for saving the presentation
        n :     int
                number of function to plot at x axis
        j :     int
                number of function to plot at y axis
        h :     float
                step for numerical solution
        step :  int
                set step in frames for animation
        """
        x = self.runge_4(tn, h)[1]
        fig = plt.figure()
        ax = fig.add_subplot()
        ax.set_xlim(min(x[n]), max(x[n]))
        ax.set_ylim(min(x[j]), max(x[j]))
        (line,) = ax.plot([], [])
        (marker,) = ax.plot([], [], marker="o", markersize=10)
        ax.set_xlabel(f"x[{n}]")
        ax.set_ylabel(f"x[{j}]")
        ax.set_title("Phase plane")
        step = 500

        def animate(i):
            line.set_data(x[n, : step * i], x[j, : step * i])
            marker.set_data(x[n, step * i], x[j, step * i])
            return line, marker

        frs = int(np.floor(x[0].size / step))
        anim = FuncAnimation(fig, animate, frames=frs, interval=0.001)
        anim.save(f"{filename}.gif", writer="pillow")

    def phase_3d_anim(
        self,
        tn: Value,
        filename: str,
        n: int = 0,
        j: int = 1,
        k: int = 2,
        h: Value = 0.001,
        step: int = 500,
    ):
        """
        Method for creation of the animation of the numerical solution of the equation in phase space
        ---
        Parameters
        tn :    float | int
                end time point
        filename: str
                name of the file for saving the presentation
        n :     int
                number of function to plot at x axis
        j :     int
                number of function to plot at y axis
        k :     int
                number of function to plot at z axis
        h :     float
                step for numerical solution
        step :  int
                set step in frames for animation
        """
        x = self.runge_4(tn, h)[1]
        fig = plt.figure()
        ax = fig.add_subplot(projection="3d")
        ax.set_xlim3d(min(x[n]), max(x[n]))
        ax.set_ylim3d(min(x[j]), max(x[j]))
        ax.set_zlim3d(min(x[k]), max(x[k]))
        (line,) = ax.plot([], [], [])
        (marker,) = ax.plot([], [], [], marker="o", markersize=10)
        ax.set_xlabel(f"x[{n}]")
        ax.set_ylabel(f"x[{j}]")
        ax.set_zlabel(f"x[{k}]")
        ax.set_title("Phase space")

        def animate(i):
            line.set_data_3d(x[n, : step * i], x[j, : step * i], x[k, : step * i])
            marker.set_data_3d(x[n, step * i], x[j, step * i], x[k, step * i])
            return line, marker

        frs = int(np.floor(x[0].size / step))
        anim = FuncAnimation(fig, animate, frames=frs, interval=0.001)
        anim.save(f"{filename}.gif", writer="pillow")

# c_1 = 1/9; 0.1; 0.05;
# c_2 = 1; 0.1; 2; 5; 15;
# R = 1.4; 1.2; 1.8; 1.9;
# ga = -0.8; -0.5; 0.1; 0.01;
# gb = -0.5; -0.3; -0.1;


inductivity = 1 / 7
c_1 = 1 / 9
c_2 = 1
R = 1.4
E = 100
ga = -0.8
gb = -0.5


def g_peace(x: Value) -> float:
        return gb * x + 0.5 * (ga - gb) * (abs(x + E) - abs(x - E))


def eq_1(t: Value, X: Vector) -> float:
        return ((X[1] - X[0]) / R - g_peace(X[0])) / c_1


def eq_2(t: Value, X: Vector) -> float:
        return ((X[0] - X[1]) / R + X[2]) / c_2


def eq_3(t: Value, X: Vector) -> float:
        return -X[1] / inductivity


a = ODEsystem(0, [10, 0, 0], (eq_1, eq_2, eq_3))
# a.phase_3d(50)
# a.plot_solution(100)
# a.phase_anim(50, "Chua_2d")
a.phase_3d_anim(50, "Chua_3d")
# a.phase_portret(10, 0, 1, np.linspace(1, 10, 11), (0, 5, 10))